In [1]:
##DATA INGESTION and DATA WRANGLING

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)
import seaborn as sns
import re
import json
import requests
import subprocess
import glob
import os

In [3]:
#!pip install geopandas
#!pip install pygeocoder
#!pip install uszipcode

In [4]:
import geopandas as gpd
from geopandas import GeoDataFrame
from pygeocoder import Geocoder

/Applications/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [5]:
# get data: Towing Report from Montgomery Data 
#towing = pd.DataFrame(requests.get('https://data.montgomerycountymd.gov/resource/i6vn-3s6e.json').json()) only allows first 1000 rows
url = 'https://data.montgomerycountymd.gov/api/views/i6vn-3s6e/rows.csv?accessType=DOWNLOAD'
towing = pd.read_csv(url)
towing.head()

,Tow Date,Vehicle Year,Vehicle Make,Vehicle Model,Tow Reason,Storage Company,Notes,Incident Date,Location,City,Geo-location field
0,04/30/2020,2019.0,JEEP,NaN,NOP,TCL,NaN,05/02/0020,19757 CRYSTAL ROCK DR,GERMANTOWN,POINT (-77.261484 39.179888)
1,06/13/2020,2015.0,JEEP,GCH,FIRE,TCL,NaN,06/15/0020,20107 TORREY POND PL,MONTGOMERY VILLAGE,POINT (-77.204705 39.185963)
2,05/21/2020,2005.0,GMC,NaN,FIRE,tcl,FIRE LANE // HENRYS 301-927-0007 // TCL,05/23/0020,6660 EAMES WAY,BETHESDA,NaN
3,04/17/2020,2015.0,HYUN,ACC,NOP,A&M,NOP A&M 301-421-0953,04/19/0020,100 BLK BOWSPRIT CT,GAITHERSBURG,NaN
4,05/24/2020,2015.0,MINI,COOPER,NOP,tcl,NO PERMIT // AUBURN // 301-977-2740 // TCL,05/26/0020,ARIZONA CIR / WESTLAKE DR,BETHESDA,NaN


In [6]:
#Rename columns
towing.columns = ["tow_date", "vehicle_year", "vehicle_make", "vehicle_model", "tow_reason", "storage_company", "notes", "incident_date", "street", "city", "geo_location"]
        

In [7]:
#capitalize on first letter of each string in towing.notes, towing.story_company, towing.location, towing.city
towing.loc[:,"notes"] = towing.loc[:,"notes"].str.title()
towing.loc[:,"storage_company"] = towing.loc[:,"storage_company"].str.title()
towing.loc[:,"street"] = towing.loc[:,"street"].str.title()
towing.loc[:,"city"] = towing.loc[:,"city"].str.title()
towing.head()
towing.dtypes

tow_date            object
vehicle_year       float64
vehicle_make        object
vehicle_model       object
tow_reason          object
storage_company     object
notes               object
incident_date       object
street              object
city                object
geo_location        object
dtype: object

In [8]:
#drop columns not needed
towing.drop('vehicle_year', axis=1, inplace=True) 
towing.drop('vehicle_make', axis=1, inplace=True)
towing.drop('vehicle_model', axis=1, inplace=True)

In [9]:
#remove apostrophe in notes and storage company before capitalizing with str.title
towing["notes"] = towing["notes"].str.replace("'", "")   #remove apostrophe
towing["storage_company"] = towing["storage_company"].str.replace("'", "")   #remove apostrophe

#capitalize on first letter of each string in towing.notes, towing.storage_company, towing.city
towing.loc[:,"storage_company"] = towing.loc[:,"storage_company"].str.title()
towing.loc[:,"city"] = towing.loc[:,"city"].str.title()

#other string fixes
towing["tow_reason"] = towing["tow_reason"].str.upper() 
towing["storage_company"] = towing["storage_company"].str.replace("Rickys ", "Rickies") #fix mispelling
towing["notes"] = towing["notes"].str.replace("Rickys ", "Rickies")
towing["storage_company"] = towing["storage_company"].str.replace("HenryS", "Henrys") #fix mispelling
towing["notes"] = towing["notes"].str.replace("HenryS", "Henrys")
towing["notes"] = towing["notes"].str.replace("HenryS", "Henrys")
towing["notes"] = towing["notes"].str.replace("Herny'S", "Henrys")
towing["notes"] = towing["notes"].str.replace("HenrS", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henrus", "Henrys")
towing["notes"] = towing["notes"].str.replace("HenrysTowing", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henry;'S", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henrsy", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henr'S", "Henrys")
towing["notes"] = towing["notes"].str.replace("Milllers", "Millers")
towing["notes"] = towing["notes"].str.replace("MillerS", "Millers")
towing["notes"] = towing["notes"].str.replace("Prestege", "Prestige")
towing["notes"] = towing["notes"].str.replace("Pretige", "Prestige")
towing["notes"] = towing["notes"].str.replace("Pretisge", "Prestige")
towing["notes"] = towing["notes"].str.replace("Presteig", "Prestige")
towing["notes"] = towing["notes"].str.replace("Prestigee", "Prestige")
towing["notes"] = towing["notes"].str.replace("Presige", "Prestige")
towing["notes"] = towing["notes"].str.replace("Prestiage", "Prestige")
towing["notes"] = towing["notes"].str.replace("Pretisge", "Prestige")
towing["notes"] = towing["notes"].str.replace("Auhtorized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Athorized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authortized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Autherized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authorizd", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authoriaed", "Authorized")
towing["notes"] = towing["notes"].str.replace("Uthorized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authorizerd", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authorixzed", "Authorized")
towing["notes"] = towing["notes"].str.replace("Autorized", "Authorized")
towing["notes"] =  towing["notes"].str.replace("RickyS", "Rickys")
towing["notes"] = towing["notes"].str.replace("Rickeys", "Rickys")
towing["notes"] = towing["notes"].str.replace("Rckys", "Rickys")
towing["notes"] = towing["notes"].str.replace("RickS", "Rickys")
towing["notes"] = towing["notes"].str.replace("Ricky", "Rickys")
towing["notes"] = towing["notes"].str.replace("SpeedyS", "Speedys")
towing["notes"] = towing["notes"].str.replace("Diviersified", "Diversified")
towing["notes"] = towing["notes"].str.replace("Deversified", "Diversified")
towing["notes"] = towing["notes"].str.replace("Diviersified", "Diversified")
towing["notes"] = towing["notes"].str.replace("Diversied", "Diversified")
towing["notes"] = towing["notes"].str.replace("A & M", "A&M")


towing["storage_company"] = towing["storage_company"].str.replace("Autherized", "Authorized")
towing["storage_company"] = towing["storage_company"].str.replace("Autorized Towing", "Authorized")
towing["storage_company"] = towing["storage_company"].str.replace("HenrysTowing", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henry", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henryss", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henreys", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henrys Towin", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("A & M", "A&M")


#convert date to datetime object
towing['tow_date'] = pd.to_datetime(towing['tow_date'])



towing.head()
print(towing.count())
towing.dtypes

tow_date           20415
tow_reason         20408
storage_company    19649
notes              19963
incident_date      20424
street             20424
city               20424
geo_location       20078
dtype: int64


tow_date           datetime64[ns]
tow_reason                 object
storage_company            object
notes                      object
incident_date              object
street                     object
city                       object
geo_location               object
dtype: object

In [10]:
#Split geo-point to retrieve latitude and longitude to store 
towing_geo = towing["geo_location"].str.split(" ", n = 5, expand = True)
print(towing_geo)

           0            1           2
0      POINT  (-77.261484  39.179888)
1      POINT  (-77.204705  39.185963)
2        NaN          NaN         NaN
3        NaN          NaN         NaN
4        NaN          NaN         NaN
...      ...          ...         ...
20981  POINT  (-76.939653  39.072248)
20982  POINT   (-77.06692  39.088854)
20983  POINT   (-77.06692  39.088854)
20984  POINT  (-76.999533  38.998597)
20985  POINT  (-77.181901  39.191003)

[20986 rows x 3 columns]


In [11]:
#create new towing dataset columns from split colums columns: "latitude", "longitude", and 
#"location" for use in FCC census block lookup
towing["longitude"] = towing_geo.iloc[:,1]
towing["longitude"] = towing["longitude"].str.replace("(", "")
towing["latitude"] = towing_geo.iloc[:,2]
towing["latitude"] = towing["latitude"].str.replace(")", "")
towing["location"] = towing["latitude"] + "," + towing["longitude"]
towing["latitude"] = towing["latitude"].astype("float")
towing["longitude"] = towing["longitude"].astype("float")
towing.head()
towing.dtypes

tow_date           datetime64[ns]
tow_reason                 object
storage_company            object
notes                      object
incident_date              object
street                     object
city                       object
geo_location               object
longitude                 float64
latitude                  float64
location                   object
dtype: object

In [12]:
#eliminate rows with Nan
#towing = towing.dropna()
#towing = towing.reset_index(drop=True)
#change text columns to string datatype
towing["notes"].astype(str)
towing["storage_company"].astype(str)
towing["city"].astype(str)
towing["tow_reason"].astype(str)
# remove special characters from notes column
towing["notes"] = towing["notes"].str.replace("//", " ")
towing["notes"] = towing["notes"].str.replace("/", " ")
towing["notes"] = towing["notes"].str.replace(':', " ")
towing["notes"] = towing["notes"].str.replace('-', "")
towing["notes"] = towing["notes"].str.replace('#', "")
towing["notes"] = towing["notes"].str.replace(".", "")
towing["notes"] = towing["notes"].str.replace(",", "")
towing["notes"] = towing["notes"].str.replace('@', "")
towing["notes"] = towing["notes"].str.replace("Rickys", "Rickies")
towing["notes"] = towing["notes"].str.replace("Tcl", "")

# Use regex techniques on notes column
towing['notes'] = towing['notes'].str.replace(r'\d+','')
towing["notes"] = towing["notes"].str.replace('Repo ', "")
towing["notes"] = towing["notes"].str.replace('Walk Off ', "")
towing["notes"] = towing["notes"].str.replace('Nop ', "")
towing["notes"] = towing["notes"].str.replace("Nop", "")
towing["notes"] = towing["notes"].str.replace('Fire Lane ', "")
towing["notes"] = towing["notes"].str.replace('Vin ', "")
towing["notes"] = towing["notes"].str.replace('Fin. ', "")
towing["notes"] = towing["notes"].str.replace('Prv ', "")
towing["notes"] = towing["notes"].str.replace("No Permit ", "")
towing["notes"] = towing["notes"].str.replace('Unauthorized ', "")
towing["notes"] = towing["notes"].str.replace('Unautorized', "")
towing["notes"] = towing["notes"].str.replace('Parking ', "")
towing["notes"] = towing["notes"].str.replace('Pkg', "")
towing["notes"] = towing["notes"].str.replace('Management ', "")
towing["notes"] = towing["notes"].str.replace('Mgmt', "")
towing["notes"] = towing["notes"].str.replace('Mgt ', "")
towing["notes"] = towing["notes"].str.replace('Request ', "")
towing["notes"] = towing["notes"].str.replace('Req ', "")
towing["notes"] = towing["notes"].str.replace('Final ', "")
towing["notes"] = towing["notes"].str.replace('Notice ', "")
towing["notes"] = towing["notes"].str.replace('No Permit ', "")
towing["notes"] = towing["notes"].str.replace('Invalid Permit ', "")
towing["notes"] = towing["notes"].str.replace("Expired Permit ", "")
towing["notes"] = towing["notes"].str.replace("Exp Permit ", "")
towing["notes"] = towing["notes"].str.replace("Btl ", "")
towing["notes"] = towing["notes"].str.replace("Rickys ", "Rickies")
towing["notes"] = towing["notes"].str.replace("No Commercial Vehs", "")
towing["notes"] = towing["notes"].str.replace("Parked In Reserved Spot", "")
towing["notes"] = towing["notes"].str.replace("Parked In Handicap Without Permit", "")
towing["notes"] = towing["notes"].str.replace("No Overnight ", "")
towing["notes"] = towing["notes"].str.replace("Handicapped ", "")
towing["notes"] = towing["notes"].str.replace("Taking Two Spaces ", "")
towing["notes"] = towing["notes"].str.replace("Reserved Space ", "")
towing["notes"] = towing["notes"].str.replace("Marked Space ", "")
towing["notes"] = towing["notes"].str.replace("Back To The Lot ", "") 
towing["notes"] = towing["notes"].str.replace("()", "")
towing["notes"] = towing["notes"].str.replace("Displayed ", "")
towing["notes"] = towing["notes"].str.replace("Parked In", "")
towing["notes"] = towing["notes"].str.replace("Double Parked", "")
towing["notes"] = towing["notes"].str.replace("No Zone", "")
towing["notes"] = towing["notes"].str.replace("No Permit ", "")
towing["notes"] = towing["notes"].str.replace("Unreadable", "") 
towing["notes"] = towing["notes"].str.replace("Blocked", "")
towing["notes"] = towing["notes"].str.replace("Per Mgmnt", "")
towing["notes"] = towing["notes"].str.replace("Blk Access", "")
towing["notes"] = towing["notes"].str.replace("Visitor Spot Over Time Limit", "")
towing["notes"] = towing["notes"].str.replace("On Private Property", "")
towing["notes"] = towing["notes"].str.replace("Over Time Limit For Visitor", "")
towing["notes"] = towing["notes"].str.replace("Illegally Parked", "")
towing["notes"] = towing["notes"].str.replace("Expired Permit", "")
towing["notes"] = towing["notes"].str.replace("Blocking Access", "")
towing["notes"] = towing["notes"].str.replace("Covered", "")

towing["notes"] = towing["notes"].str.strip()

towing.tail()

,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,longitude,latitude,location
20981,2021-02-16,PKG,Tcl,Henrys Fire Ln,02/18/0021,3536 Gentry Ridge Ct,Silver Spring,POINT (-76.939653 39.072248),-76.939653,39.072248,"39.072248,-76.939653"
20982,2021-02-10,NOP,Authorized,Authorized,10/23/0002,14120 Weeping Willow Dr,Silver Spring,POINT (-77.06692 39.088854),-77.066920,39.088854,"39.088854,-77.06692"
20983,2021-02-24,PKG,Tcl,Authorized,02/26/0021,14120 Weeping Willow Dr,Silver Spring,POINT (-77.06692 39.088854),-77.066920,39.088854,"39.088854,-77.06692"
20984,2021-01-15,EXP,Drive,Drive,01/17/0021,8658 Piney Branch Rd,Silver Spring,POINT (-76.999533 38.998597),-76.999533,38.998597,"38.998597,-76.999533"
20985,2021-02-14,REPO,Tcl,Santander Notic,02/16/0021,8847 Thomas Lea Ter,Montgomery Village,POINT (-77.181901 39.191003),-77.181901,39.191003,"39.191003,-77.181901"


In [13]:
#create list of company names ocurring 
company_count = towing["storage_company"].value_counts()
company_freq = towing["storage_company"].value_counts(normalize = True)
company = pd.Series(towing["storage_company"])
names = company.drop_duplicates()
top_names = names
#print(company_count)
#print(company_freq)
print(top_names)
len(towing)

0                                 Tcl
3                                 A&M
5                              Auburn
6                             Big Tow
7                                 Trp
                     ...             
20169    Flowmax Towing And Transport
20409                               H
20481                   14505 Lee Hwy
20658                            Avis
20761                             Cgi
Name: storage_company, Length: 276, dtype: object


20986

In [14]:
#Finish cleaning Storage Company


word_set = set(top_names)
x=-1
rowcount = len(towing)-1
while x < rowcount :
    x = x+1
    if towing.iloc[x,2] == "Tcl" :
        string_to_be_searched = str(towing.iloc[x,3])
        for w in string_to_be_searched.split():
            if w in word_set :
                towing.iloc[x,2] = w
                print("%s in string" % w)    
            else :
                print("not found ", w)
    else :
        print(x, "is ok")
        



       

not found  nan
not found  nan
Henrys in string
3 is ok
Auburn in string
5 is ok
6 is ok
7 is ok
Auburn in string
9 is ok
Grace in string
not found  Towing
not found  nan
12 is ok
Auburn in string
14 is ok
not found  nan
A&M in string
not found  Inv
not found  Permit
Henrys in string
18 is ok
not found  Reserved
Auburn in string
Authorized in string
21 is ok
Henrys in string
Henrys in string
24 is ok
Prestige in string
Auburn in string
27 is ok
Henrys in string
29 is ok
30 is ok
not found  Illegally
not found  Pkd
not found  Pkd
not found  Overnight
Custom in string
Henrys in string
Henrys in string
34 is ok
Henrys in string
Authorize in string
Prestigue in string
not found  Towing
Authorized in string
not found  Towing
not found  To
not found  For
not found  No
not found  Zone
Authorized in string
40 is ok
41 is ok
42 is ok
43 is ok
44 is ok
45 is ok
46 is ok
not found  Handicap
not found  W
not found  O
not found  Tag
not found  Or
not found  Permit
Authorized in string
Led in string


Henrys in string
478 is ok
not found  Altered
not found  Permit
Authorized in string
not found  Space
not found  W
not found  O
not found  Permit
Henrys in string
Prestige in string
not found  Prop
482 is ok
Auburn in string
not found  In
not found  Resrvd
not found  Spot
Bigtow in string
Speedy in string
not found  ()
not found  Reserved
not found  Pking
Henrys in string
Henrys in string
Henrys in string
489 is ok
490 is ok
Authorized in string
not found  Unable
not found  To
not found  Provide
492 is ok
493 is ok
Custom in string
not found  Towing
Prestige in string
not found  Towing
not found  Private
not found  Property
496 is ok
497 is ok
Henrys in string
499 is ok
500 is ok
501 is ok
502 is ok
Henrys in string
Drive in string
Authorized in string
506 is ok
507 is ok
508 is ok
509 is ok
Authorized in string
511 is ok
512 is ok
not found  Blocking
not found  The
not found  Alleyway
Henrys in string
Henrys in string
515 is ok
Speedy in string
not found  Reserved
Auburn in string
not

Prestige in string
Authorized in string
Authorized in string
1046 is ok
1047 is ok
Authorized in string
not found  Towing
Henrys in string
1050 is ok
1051 is ok
Drive in string
Prestige in string
Henrys in string
Henrys in string
Authorize in string
not found  Reason
Diversified in string
Authorized in string
1059 is ok
1060 is ok
Henrys in string
Prestigue in string
not found  No
not found  Perm
Authorized in string
1064 is ok
Authorized in string
1066 is ok
1067 is ok
not found  Dbl
not found  Parked
Henrys in string
1069 is ok
1070 is ok
Authorized in string
not found  Flat
not found  Tire
not found  ()
Henrys in string
Authorized in string
Auth in string
Prestige in string
not found  Pp
not found  No
not found  Perm
Authorized in string
1077 is ok
Authorized in string
not found  In
not found  A
Henrys in string
1080 is ok
1081 is ok
1082 is ok
Authorized in string
Rickies in string
1085 is ok
Henrys in string
Henrys in string
1088 is ok
1089 is ok
Prestige in string
Henrys in strin

Authorized in string
not found  Abnd
Millers in string
not found  Towing
not found  And
not found  Recovery
Authorized in string
not found  T
not found  Cl
Henrys in string
1530 is ok
Henrys in string
not found  Property
1532 is ok
not found  No
not found  Visitor
not found  Placard
Henrys in string
Authorized in string
not found  Owner
not found  Notif
Henrys in string
not found  Henry
not found  Call
not found  In
not found  WalkOff
Auburn in string
Henrys in string
1539 is ok
Henrys in string
1541 is ok
Henrys in string
1543 is ok
not found  In
Henrys in string
Authorized in string
Led in string
not found  Towing
not found  ()
1547 is ok
1548 is ok
not found  Walkoff
Millers in string
1550 is ok
Authorized in string
not found  Owner
not found  Notified
1552 is ok
1553 is ok
1554 is ok
not found  Reserved
Big in string
not found  Tow
not found  Expired
not found  Tags
RickiesTowing in string
not found  Allied
not found  Bank
not found  No
not found  P
Henrys in string
not found  Back

Prestigue in string
Speedy in string
Authorized in string
Authorized in string
not found  No
not found  P
2096 is ok
2097 is ok
2098 is ok
not found  Reserved
Millers in string
not found  Towing
Henrys in string
2101 is ok
2102 is ok
Henrys in string
not found  Unk
not found  Year
2104 is ok
2105 is ok
Henrys in string
not found  In
not found  Resident
not found  Space
Speedy in string
2108 is ok
2109 is ok
2110 is ok
2111 is ok
Speedy in string
2113 is ok
2114 is ok
2115 is ok
A&M in string
not found  Towing
2117 is ok
2118 is ok
not found  LoftonS
not found  Towing
not found  Financial
not found  Co
not found  Darcars
not found  Toyt
not found  Of
not found  Silver
not found  Spring
2120 is ok
2121 is ok
not found  In
Authorized in string
not found  Toyt
not found  Financial
Greenwood in string
A&M in string
Henrys in string
not found  Owner
not found  Notif
Custom in string
2127 is ok
Henrys in string
not found  nan
Henrys in string
Henrys in string
2132 is ok
Millers in string
not 

...

Avis in string
not found  Budget
not found  Group
Fast in string
not found  Guys
not found  Towing
Drive in string
not found  Motorcycle
not found  Spaces
Big in string
not found  Tow
18841 is ok
18842 is ok
Henrys in string
18844 is ok
Henrys in string
Authorized in string
not found  Towing
Fire in string
not found  Lane
Santander in string
18848 is ok
not found  Allie
not found  Financial
not found  Parr
not found  Services
Auburn in string
not found  Unauth
not found  Parking
18851 is ok
Auth in string
Fire in string
not found  Lane
Henrys in string
Henrys in string
not found  Reserved
Henrys in string
Authorized in string
not found  Visitor
not found  Exp
Rickies in string
Henrys in string
Big in string
not found  Tex
not found  Trailer
not found  On
not found  The
not found  Run
not found  Towing
18860 is ok
Prestige in string
18862 is ok
18863 is ok
18864 is ok
not found  Visitor
not found  In
not found  Resd
not found  Space
Drive in string
not found  American
not found  Honda
n

Speedy in string
not found  Cmax
not found  S
not found  Private
not found  Property
Prestige in string
Big in string
not found  Tow
19290 is ok
Prestige in string
Rbf in string
not found  Towing
not found  ()
Drive in string
not found  Towing
19294 is ok
19295 is ok
not found  Va
not found  Temp
not found  Tag
Henrys in string
Speedy in string
Auburn in string
not found  Rickiess
not found  No
not found  Visiots
Aubrun in string
not found  Toyt
not found  Financial
Henrys in string
not found  Wells
not found  Fargo
Etc in string
not found  Recovery
not found  Reason
Henrys in string
Drive in string
not found  Handicap
not found  W
not found  No
not found  Placard
19306 is ok
Henrys in string
19308 is ok
not found  Ifo
not found  Dumpster
Diversified in string
not found  Rickiess
not found  Back
not found  To
not found  Lot
Henrys in string
not found  No
not found  P
not found  Parked
not found  Over
not found  Time
not found  Limit
Rickies in string
19313 is ok
19314 is ok
Authorized 

Henrys in string
not found  Dbl
not found  Pkd
not found  And
19806 is ok
Henrys in string
19808 is ok
Drive in string
19810 is ok
Prestige in string
Prestige in string
Auburn in string
not found  Towing
19814 is ok
19815 is ok
Authorized in string
Henrys in string
not found  Dbl
19818 is ok
not found  On
not found  The
not found  Run
not found  Towing
not found  ()
19820 is ok
not found  Community
not found  Loans
not found  Of
not found  America
19822 is ok
19823 is ok
Henrys in string
19825 is ok
19826 is ok
Henrys in string
not found  Private
not found  Property
Prestige in string
19829 is ok
19830 is ok
19831 is ok
Henrys in string
Henrys in string
not found  Towing
not found  To
not found  For
not found  (No
not found  Zone)
Speedy in string
not found  Towing
not found  Resd
not found  In
not found  Visitor
not found  Space
not found  Tag
not found  DoesnT
not found  Match
not found  Permit
Drive in string
Henrys in string
19837 is ok
Prestige in string
Grace in string
not found 

Big in string
not found  Tow
20362 is ok
Auburn in string
20364 is ok
20365 is ok
not found  Ally
not found  Owner
not found  Adr
not found  Same
not found  Ally
not found  Sarah
not found  Hogs
not found  Roby
not found  Ter
not found  Reserved
Prestige in string
Authorized in string
not found  Towing
20370 is ok
Henrys in string
Authorized in string
20373 is ok
20374 is ok
not found  Reserve
not found  Space
Prestigue in string
20376 is ok
20377 is ok
20378 is ok
Speedy in string
not found  Towing
Millers in string
Henrys in string
not found  To
Henrys in string
Henrys in string
Henrys in string
not found  Credit
not found  Acceptance
Cti in string
not found  Owner
not found  Adr
not found  Is
not found  Russel
not found  Ave
not found  Mt
not found  Rainer
not found  Md
not found  Comunity
20387 is ok
not found  *RepoD*
not found  Lender
not found  Credit
not found  Acceptance
not found  ()
not found  Tow
not found  Co
Cti in string
not found  Towing
not found  ()
not found  Titlema

Speedy in string
20835 is ok
Henrys in string
Authorized in string
20838 is ok
20839 is ok
20840 is ok
20841 is ok
Authorized in string
not found  Towing
not found  Credit
not found  Acceptance
Cti in string
Fire in string
Speedy in string
20845 is ok
20846 is ok
not found  Repossession
not found  Flex
Drive in string
not found  Svcs
not found  Back
not found  To
not found  Lot
Auburn in string
20849 is ok
Rickies in string
20851 is ok
Henrys in string
not found  Back
not found  To
not found  Lot
20853 is ok
Authorized in string
not found  Towing
20855 is ok
20856 is ok
Henrys in string
Rbf in string
not found  Towing
not found  ()
Bigtow in string
Henrys in string
not found  No
not found  Prkg
not found  Zone
Authorized in string
not found  Towing
20862 is ok
Henrys in string
not found  Walkoff
20864 is ok
Authorized in string
Authorized in string
20867 is ok
20868 is ok
20869 is ok
20870 is ok
20871 is ok
Henrys in string
not found  Unauthorizedparking
not found  And
not found  Expir

In [15]:
#Check Nulls and Replace storage company Tcl and Nan with "Other" as company name where we can
print(towing.isna().sum())
towing["storage_company"] = towing["storage_company"].str.replace("NaN", "Other")
towing["storage_company"] = towing["storage_company"].str.replace("Tcl", "Other")
print(towing.isna().sum())  #string replace still showing as Nan?
#Check Nulls and generage missing "location" long/lat pairs from address
towing = towing.dropna(subset = ["street", "city", "incident_date"]) #delete 562 na values to use lon/lat generator
print(towing.isna().sum()) #leaves 300+ lon/lat to be generated from address info minus zip code

tow_date            571
tow_reason          578
storage_company    1337
notes              1023
incident_date       562
street              562
city                562
geo_location        908
longitude           908
latitude            908
location            908
dtype: int64
tow_date            571
tow_reason          578
storage_company    1337
notes              1023
incident_date       562
street              562
city                562
geo_location        908
longitude           908
latitude            908
location            908
dtype: int64
tow_date             9
tow_reason          16
storage_company    775
notes              461
incident_date        0
street               0
city                 0
geo_location       346
longitude          346
latitude           346
location           346
dtype: int64


In [16]:
#Looked at geocoding:  Cannot generate missing lon/lat from address information; only crossstreets not street address
#drop 346 geocode missing rows
towing = towing.dropna(subset = ["location", "geo_location", "latitude", "longitude"])
#print(towing.isna().sum())
len(towing)

20078

In [17]:
#Write csv files to local drive for processing FCC Block extraction
max_row = len(towing)
batch1 = towing[0:10000]
batch2 = towing[10000:20000]
batch3 = towing[20000:20078]
#write to csv - 3 batches
batch1.to_csv('/Users/elizabethclick/Desktop/DATA205/TowBatch1.csv', index = False)
batch2.to_csv('/Users/elizabethclick/Desktop/DATA205/TowBatch2.csv', index = False)
batch3.to_csv('/Users/elizabethclick/Desktop/DATA205/TowBatch3.csv', index = False)
#check batch length
print(len(batch1))
print(len(batch2))
print(len(batch3))
print(max_row)

10000
10000
78
20078


In [18]:
#process in short bursts
process_chunksize = 10
#Initialize separated dataframe "towing" saved on drive 
os.chdir('/Users/elizabethclick/Desktop/DATA205')
mainCSV1 = 'TowBatch1.csv'
mainCSV2 = 'TowBatch2.csv'
mainCSV3 = 'TowBatch3.csv'

mainDF1 = pd.read_csv(mainCSV1, chunksize=process_chunksize)
mainDF2 = pd.read_csv(mainCSV2, chunksize=process_chunksize)
mainDF3 = pd.read_csv(mainCSV3, chunksize=process_chunksize)

In [19]:
#Pull desired census data from Census API into tractDF dataframe
census_api_url='https://api.census.gov/data/2019/acs/acs5?key=5533410471151873cdf3e32fd6cacb45753daa2a&get=NAME,B29004_001E,B01003_001E,B02001_003E&for=block%20group:*&in=state:24&in=county:031&in=tract:*'
responseCensus = requests.get(census_api_url, headers={'Content-Type': 'application/json'})
textCensus=json.loads(responseCensus.content)
tractDF = pd.DataFrame(textCensus)
print('Shape of census tract file: ' + str(tractDF.shape))

tractDF.rename(columns = tractDF.iloc[0], inplace=True)
tractDF = tractDF.iloc[1:]
tractDF.rename(index=str,columns={'B29004_001E':'hh_income','B01003_001E':'pop_total',
                                  'B02001_003E':'pop_black'}, inplace=True)

Shape of census tract file: (615, 8)


In [20]:
#Here is the tractDF with demographics to join with dataMontgomery towing set
tractDF.head()

,NAME,hh_income,pop_total,pop_black,state,county,tract,block group
1,"Block Group 4, Census Tract 7032.08, Montgomer...",158125,1646,319,24,031,703208,4
2,"Block Group 4, Census Tract 7015.09, Montgomer...",56786,3041,1284,24,031,701509,4
3,"Block Group 2, Census Tract 7015.08, Montgomer...",68750,590,263,24,031,701508,2
4,"Block Group 1, Census Tract 7039.01, Montgomer...",160938,1553,354,24,031,703901,1
5,"Block Group 2, Census Tract 7057.02, Montgomer...",134000,1697,21,24,031,705702,2


In [21]:
tract2 = tractDF.sort_values(by='tract')
tract2.head()

,NAME,hh_income,pop_total,pop_black,state,county,tract,block group
97,"Block Group 1, Census Tract 7001.01, Montgomer...",154762,1922,184,24,031,700101,1
158,"Block Group 3, Census Tract 7001.01, Montgomer...",135208,800,10,24,031,700101,3
159,"Block Group 2, Census Tract 7001.01, Montgomer...",208929,1931,39,24,031,700101,2
494,"Block Group 3, Census Tract 7001.03, Montgomer...",172778,3112,345,24,031,700103,3
176,"Block Group 2, Census Tract 7001.03, Montgomer...",218173,1017,193,24,031,700103,2


In [22]:
tractDF['tract'][0]
tractDF.dtypes

NAME           object
hh_income      object
pop_total      object
pop_black      object
state          object
county         object
tract          object
block group    object
dtype: object

In [23]:
#concatenate tract and block codes into one column
tractDF['block group'] = tractDF['block group'].apply(str)
tractDF['tract'] = tractDF['tract'].apply(str)

for i in range(0,len(tractDF)):
    tractDF['block group'][i] = tractDF['tract'][i] + tractDF['block group'][i]
    
print(tractDF.head())


                                                NAME hh_income pop_total  \
1  Block Group 4, Census Tract 7032.08, Montgomer...    158125      1646   
2  Block Group 4, Census Tract 7015.09, Montgomer...     56786      3041   
3  Block Group 2, Census Tract 7015.08, Montgomer...     68750       590   
4  Block Group 1, Census Tract 7039.01, Montgomer...    160938      1553   
5  Block Group 2, Census Tract 7057.02, Montgomer...    134000      1697   

  pop_black state county   tract block group  
1       319    24    031  703208     7032084  
2      1284    24    031  701509     7015094  
3       263    24    031  701508     7015082  
4       354    24    031  703901     7039011  
5        21    24    031  705702     7057022  


In [24]:
for column in tractDF.columns:
    print(column)
tractDF.dtypes

NAME
hh_income
pop_total
pop_black
state
county
tract
block group


NAME           object
hh_income      object
pop_total      object
pop_black      object
state          object
county         object
tract          object
block group    object
dtype: object

In [25]:
#Define tract finding function using FCC API with longitude and latitudes used to add block codes to towing dataset
def getblock(lonlat):
    block = -999
    api_url='https://geo.fcc.gov/api/census/area?lat='+str(lonlat[0])+'&lon='+str(lonlat[1])+'&format=json'
    headers = {'Content-Type': 'application/json'}
    response = requests.get(api_url, headers=headers)
    try:
        block=json.loads(response.content.decode('utf-8'))['results'][0]['block_fips'][5:12]
    except:
        print('No block found from FCC geo API')
    return block

In [56]:
#process first 10000 rows of towing dataframe - get block codes in a new column
df1 = pd.DataFrame()
itld=0
itlu=1
    
for mainDF_chunk in mainDF1:
            nLinesMainCSVd=itld*process_chunksize
            nLinesMainCSVu=itlu*process_chunksize
            print('New group from main DF, lines '+str(nLinesMainCSVd) + ' to ' + str(nLinesMainCSVu))

            mainDF_chunk['location']=mainDF_chunk['location'].map(lambda x: eval(str(x)))
            print('Dimension of main DF group:' + str(mainDF_chunk.shape))
    
            # Add block from FCC geo to towing dataset in new column "block"
            mainDF_chunk['block']=mainDF_chunk.location.map(lambda x: getblock(x))
            print(mainDF_chunk["block"])
           
            df1 = df1.append(mainDF_chunk)
    
            itlu+=1
            itld+=1

New group from main DF, lines 0 to 10
Dimension of main DF group:(10, 11)
0    7008182
1    7008102
2    7048033
3    7018001
4    7003092
5    7007171
6    7007191
7    7007162
8    7017041
9    7008182
Name: block, dtype: object
New group from main DF, lines 10 to 20
Dimension of main DF group:(10, 11)
10    7026022
11    7014141
12    7007212
13    7007131
14    7007233
15    7008153
16    7007212
17    7008193
18    7008161
19    7009041
Name: block, dtype: object
New group from main DF, lines 20 to 30
Dimension of main DF group:(10, 11)
20    7007151
21    7008111
22    7007232
23    7007111
24    7014222
25    7026022
26    7025001
27    7008131
28    7008134
29    7006141
Name: block, dtype: object
New group from main DF, lines 30 to 40
Dimension of main DF group:(10, 11)
30    7032141
31    7014221
32    7015051
33    7048033
34    7014222
35    7008172
36    7032132
37    7008193
38    7008113
39    7014172
Name: block, dtype: object
New group from main DF, lines 40 to 50
Dime

330    7016011
331    7012164
332    7014233
333    7021011
334    7014173
335    7008182
336    7008182
337    7016022
338    7014211
339    7008182
Name: block, dtype: object
New group from main DF, lines 340 to 350
Dimension of main DF group:(10, 11)
340    0016001
341    7008102
342    7014232
343    7007232
344    7008172
345    7032162
346    7002072
347    7007101
348    7015094
349    7038001
Name: block, dtype: object
New group from main DF, lines 350 to 360
Dimension of main DF group:(10, 11)
350    7008151
351    7032141
352    7014232
353    7020003
354    7032202
355    7008111
356    7014222
357    7044012
358    7032091
359    7006141
Name: block, dtype: object
New group from main DF, lines 360 to 370
Dimension of main DF group:(10, 11)
360    7025004
361    7007211
362    7007101
363    7032091
364    7060123
365    7007212
366    7008172
367    7014222
368    7016011
369    7003083
Name: block, dtype: object
New group from main DF, lines 370 to 380
Dimension of main DF

660    8073052
661    7007202
662    7008221
663    7032201
664    7032152
665    7017033
666    7003102
667    7015051
668    7025004
669    7008342
Name: block, dtype: object
New group from main DF, lines 670 to 680
Dimension of main DF group:(10, 11)
670    7007131
671    7012182
672    7014172
673    7032092
674    7017021
675    7025001
676    7003092
677    7016011
678    7001051
679    7012164
Name: block, dtype: object
New group from main DF, lines 680 to 690
Dimension of main DF group:(10, 11)
680    7015094
681    7015051
682    7015051
683    7008134
684    7032141
685    7008151
686    7003041
687    7033021
688    7008162
689    7008134
Name: block, dtype: object
New group from main DF, lines 690 to 700
Dimension of main DF group:(10, 11)
690    7002072
691    7007111
692    7007193
693    7014222
694    7008321
695    7008134
696    7008191
697    7012011
698    7009031
699    7032201
Name: block, dtype: object
New group from main DF, lines 700 to 710
Dimension of main DF

990    7008151
991    7032091
992    7001051
993    7008221
994    7007104
995    7023023
996    7007212
997    7017012
998    7035011
999    7007233
Name: block, dtype: object
New group from main DF, lines 1000 to 1010
Dimension of main DF group:(10, 11)
1000    7009041
1001    7027001
1002    7035021
1003    7015051
1004    7008301
1005    7032131
1006    7008133
1007    7032141
1008    7032131
1009    7009041
Name: block, dtype: object
New group from main DF, lines 1010 to 1020
Dimension of main DF group:(10, 11)
1010    7014222
1011    7007212
1012    7032143
1013    7032131
1014    7016011
1015    7007043
1016    7008102
1017    7002072
1018    7008112
1019    7014101
Name: block, dtype: object
New group from main DF, lines 1020 to 1030
Dimension of main DF group:(10, 11)
1020    7032141
1021    7057013
1022    7017033
1023    7060123
1024    7032133
1025    7014221
1026    7033021
1027    7032141
1028    7001051
1029    7012182
Name: block, dtype: object
New group from main DF, l

1300    7003091
1301    7014222
1302    7003092
1303    7014233
1304    7025001
1305    7025001
1306    7038001
1307    7024023
1308    7048062
1309    7010011
Name: block, dtype: object
New group from main DF, lines 1310 to 1320
Dimension of main DF group:(10, 11)
1310    7016022
1311    7007133
1312    7038001
1313    7015052
1314    7015032
1315    7003091
1316    7032071
1317    7021011
1318    7032202
1319    7017033
Name: block, dtype: object
New group from main DF, lines 1320 to 1330
Dimension of main DF group:(10, 11)
1320    7009041
1321    7007232
1322    7008164
1323    7032202
1324    7007193
1325    7003101
1326    7007132
1327    7012191
1328    7025001
1329    7008184
Name: block, dtype: object
New group from main DF, lines 1330 to 1340
Dimension of main DF group:(10, 11)
1330    7045013
1331    7007061
1332    7008171
1333    7007061
1334    7023012
1335    7002071
1336    7014222
1337    7007173
1338    7008181
1339    7032071
Name: block, dtype: object
New group from 

...

8740    7012212
8741    7014102
8742    7008153
8743    7048033
8744    7038001
8745    7007212
8746    7025004
8747    7032132
8748    7014103
8749    7032213
Name: block, dtype: object
New group from main DF, lines 8750 to 8760
Dimension of main DF group:(10, 11)
8750    7023023
8751    7008184
8752    7008134
8753    7038001
8754    7008172
8755    7007171
8756    7008102
8757    7038001
8758    7014211
8759    7008321
Name: block, dtype: object
New group from main DF, lines 8760 to 8770
Dimension of main DF group:(10, 11)
8760    7015051
8761    7007131
8762    7008171
8763    7008342
8764    7023011
8765    7012153
8766    7008171
8767    7008321
8768    7028001
8769    7007232
Name: block, dtype: object
New group from main DF, lines 8770 to 8780
Dimension of main DF group:(10, 11)
8770    7032133
8771    7008103
8772    7016022
8773    7032091
8774    7012153
8775    7009041
8776    7007111
8777    7032092
8778    7008321
8779    7038001
Name: block, dtype: object
New group from 

9050    7008113
9051    7003102
9052    7014103
9053    7025001
9054    7012011
9055    7012011
9056    7023021
9057    7007161
9058    7007212
9059    7008154
Name: block, dtype: object
New group from main DF, lines 9060 to 9070
Dimension of main DF group:(10, 11)
9060    7007163
9061    7034033
9062    7007233
9063    7025001
9064    7023023
9065    7007061
9066    7013151
9067    7032061
9068    7032141
9069    7038001
Name: block, dtype: object
New group from main DF, lines 9070 to 9080
Dimension of main DF group:(10, 11)
9070    7012013
9071    7008134
9072    7048033
9073    7050001
9074    7024023
9075    7032213
9076    7007132
9077    7003092
9078    7014174
9079    7007171
Name: block, dtype: object
New group from main DF, lines 9080 to 9090
Dimension of main DF group:(10, 11)
9080    7015092
9081    7023023
9082    7008111
9083    7008113
9084    7038001
9085    7016022
9086    7015051
9087    7012112
9088    7015092
9089    7001051
Name: block, dtype: object
New group from 

9360    7014222
9361    7038001
9362    7015051
9363    7008172
9364    7036011
9365    7008121
9366    7032022
9367    7014101
9368    7032163
9369    7008321
Name: block, dtype: object
New group from main DF, lines 9370 to 9380
Dimension of main DF group:(10, 11)
9370    7040001
9371    7007241
9372    7008184
9373    7038001
9374    7032162
9375    7032131
9376    7016011
9377    7032131
9378    7048062
9379    7007132
Name: block, dtype: object
New group from main DF, lines 9380 to 9390
Dimension of main DF group:(10, 11)
9380    7008221
9381    7032202
9382    7015051
9383    7008321
9384    7007061
9385    7032071
9386    7008321
9387    7028002
9388    7007131
9389    7007231
Name: block, dtype: object
New group from main DF, lines 9390 to 9400
Dimension of main DF group:(10, 11)
9390    7008134
9391    7014222
9392    7012152
9393    7008182
9394    7003041
9395    7008103
9396    7019001
9397    7014203
9398    7014211
9399    7038001
Name: block, dtype: object
New group from 

9670    7015051
9671    7021014
9672    7008154
9673    7039022
9674    7019001
9675    7019001
9676    7014102
9677    7009041
9678    7008182
9679    7015051
Name: block, dtype: object
New group from main DF, lines 9680 to 9690
Dimension of main DF group:(10, 11)
9680    7006134
9681    7032091
9682    7003041
9683    7025001
9684    7038001
9685    7032162
9686    7007131
9687    7007151
9688    7008221
9689    7007202
Name: block, dtype: object
New group from main DF, lines 9690 to 9700
Dimension of main DF group:(10, 11)
9690    7014222
9691    7012153
9692    7008182
9693    7032091
9694    7008321
9695    7025003
9696    7014144
9697    7002072
9698    7008172
9699    7060123
Name: block, dtype: object
New group from main DF, lines 9700 to 9710
Dimension of main DF group:(10, 11)
9700    7014232
9701    7032163
9702    7025001
9703    7017031
9704    7015083
9705    7008154
9706    7014174
9707    7008134
9708    7032132
9709    7008182
Name: block, dtype: object
New group from 

9980    7007101
9981    7012012
9982    7007231
9983    7017012
9984    7014222
9985    7003041
9986    7012011
9987    7025001
9988    7014172
9989    7009041
Name: block, dtype: object
New group from main DF, lines 9990 to 10000
Dimension of main DF group:(10, 11)
9990    7016011
9991    7021011
9992    7015083
9993    7035011
9994    7032143
9995    7012212
9996    7038001
9997    7016022
9998    7008102
9999    7008181
Name: block, dtype: object


In [57]:
#check csv splits to see that tables maintain all rows
df1.to_csv('/Users/elizabethclick/Desktop/DATA205/df1.csv', index = False)
df1.tail()


,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,longitude,latitude,location,block
9995,2020-10-15,NOP,Henrys,Henrys,10/17/0020,14115 Arbor Forest Dr,Rockville,POINT (-77.210356 39.087772),-77.210356,39.087772,"(39.087772, -77.210356)",7012212
9996,2020-10-14,PRV,Henrys,No Overnigt Henrys,10/16/0020,2460 Ennalls Ave,Silver Spring,POINT (-77.052599 39.04022),-77.052599,39.040220,"(39.04022, -77.052599)",7038001
9997,2020-10-23,NOP,NaN,Diversified (),10/25/0020,1424 Hampshire West Ct,Silver Spring,POINT (-76.982926 39.008008),-76.982926,39.008008,"(39.008008, -76.982926)",7016022
9998,2020-10-30,FIRE,Henrys,Henrys,11/01/0020,9808 Maple Leaf Dr,Montgomery Village,POINT (-77.201911 39.187129),-77.201911,39.187129,"(39.187129, -77.201911)",7008102
9999,2020-10-03,PKG,Other,Rickiess Towing Reserve Space,03/22/0010,19254 Circle Gate Dr,Germantown,POINT (-77.263653 39.17097),-77.263653,39.170970,"(39.17097, -77.263653)",7008181


In [27]:
#process second batch (remaining) rows of towing dataframe - get block codes in a new column
df2 = pd.DataFrame()
itld=0
itlu=1
    
for mainDF_chunk in mainDF2:
            nLinesMainCSVd=itld*process_chunksize
            nLinesMainCSVu=itlu*process_chunksize
            print('New group from main DF, lines '+str(nLinesMainCSVd) + ' to ' + str(nLinesMainCSVu))

            
            mainDF_chunk['location']=mainDF_chunk['location'].map(lambda x: eval(str(x)))
            #mainDF_chunk[['tow_date']].apply(pd.to_datetime,errors='coerce')
            print('Dimension of main DF group:' + str(mainDF_chunk.shape))
    
            # Add block from FCC geo to my main dataset in new column "block"
            mainDF_chunk['block']=mainDF_chunk.location.map(lambda x: getblock(x))
            print(mainDF_chunk["block"])
            
            df2 = df2.append(mainDF_chunk)
    
            itlu+=1
            itld+=1

New group from main DF, lines 0 to 10
Dimension of main DF group:(10, 11)
0    8073052
1    7038001
2    7008134
3    7034012
4    7014222
5    7032141
6    7008172
7    7012011
8    7017012
9    7032143
Name: block, dtype: object
New group from main DF, lines 10 to 20
Dimension of main DF group:(10, 11)
10    7032133
11    7032091
12    7012012
13    7032162
14    7008121
15    7032162
16    7032132
17    7006133
18    7003083
19    7015051
Name: block, dtype: object
New group from main DF, lines 20 to 30
Dimension of main DF group:(10, 11)
20    7007241
21    7017012
22    7032091
23    7014211
24    7038001
25    7012053
26    7007201
27    7021014
28    7032141
29    7009031
Name: block, dtype: object
New group from main DF, lines 30 to 40
Dimension of main DF group:(10, 11)
30    7032201
31    7021011
32    7008151
33    7014103
34    7014144
35    7016022
36    7008171
37    7012012
38    7008111
39    7007232
Name: block, dtype: object
New group from main DF, lines 40 to 50
Dime

330    7014222
331    7048031
332    7012135
333    7025001
334    7008103
335    7032071
336    7016011
337    7009042
338    7015083
339    7012182
Name: block, dtype: object
New group from main DF, lines 340 to 350
Dimension of main DF group:(10, 11)
340    7008151
341    7008134
342    7032092
343    7008163
344    7008221
345    7007131
346    7013131
347    7008153
348    7044031
349    7007162
Name: block, dtype: object
New group from main DF, lines 350 to 360
Dimension of main DF group:(10, 11)
350    7032092
351    7015083
352    7014222
353    7007232
354    7007131
355    7008134
356    7007133
357    7013151
358    7025001
359    7008172
Name: block, dtype: object
New group from main DF, lines 360 to 370
Dimension of main DF group:(10, 11)
360    7014233
361    7026022
362    7008173
363    7014173
364    7016022
365    7003101
366    7015083
367    7007132
368    7009011
369    7007222
Name: block, dtype: object
New group from main DF, lines 370 to 380
Dimension of main DF

660    7017041
661    7040003
662    7032131
663    7007104
664    7012011
665    7021014
666    7015083
667    7012011
668    7025004
669    7007173
Name: block, dtype: object
New group from main DF, lines 670 to 680
Dimension of main DF group:(10, 11)
670    7038001
671    7032141
672    7008132
673    7008181
674    7007201
675    7008121
676    7014222
677    7025001
678    7015051
679    7008221
Name: block, dtype: object
New group from main DF, lines 680 to 690
Dimension of main DF group:(10, 11)
680    7032123
681    7007111
682    7027001
683    7002082
684    7021014
685    7016022
686    7007132
687    7032203
688    7008194
689    7032133
Name: block, dtype: object
New group from main DF, lines 690 to 700
Dimension of main DF group:(10, 11)
690    7003101
691    7007042
692    7014173
693    7012153
694    7023012
695    7014174
696    7008134
697    7014155
698    7016022
699    7032071
Name: block, dtype: object
New group from main DF, lines 700 to 710
Dimension of main DF

990    7032132
991    7008113
992    7014102
993    7048051
994    7019001
995    7047002
996    7008321
997    7021023
998    7014211
999    7014155
Name: block, dtype: object
New group from main DF, lines 1000 to 1010
Dimension of main DF group:(10, 11)
1000    7024023
1001    7023011
1002    7032202
1003    7048041
1004    7009041
1005    7008103
1006    7014211
1007    7007132
1008    7032091
1009    7006142
Name: block, dtype: object
New group from main DF, lines 1010 to 1020
Dimension of main DF group:(10, 11)
1010    7032143
1011    7032141
1012    7014232
1013    7015051
1014    7008121
1015    7007173
1016    7020002
1017    7007132
1018    7027001
1019    7012153
Name: block, dtype: object
New group from main DF, lines 1020 to 1030
Dimension of main DF group:(10, 11)
1020    7007212
1021    7007161
1022    7007201
1023    7008221
1024    7014222
1025    7007042
1026    7014102
1027    7007232
1028    7017021
1029    7021011
Name: block, dtype: object
New group from main DF, l

1300    7009011
1301    7032092
1302    7032071
1303    7009011
1304    7009011
1305    7040003
1306    7008221
1307    7014232
1308    7026014
1309    7038001
Name: block, dtype: object
New group from main DF, lines 1310 to 1320
Dimension of main DF group:(10, 11)
1310    7012182
1311    7015051
1312    7048033
1313    7008172
1314    7008181
1315    7008172
1316    7032143
1317    7016022
1318    7008221
1319    7007212
Name: block, dtype: object
New group from main DF, lines 1320 to 1330
Dimension of main DF group:(10, 11)
1320    7032143
1321    7014174
1322    7038001
1323    7007173
1324    7007201
1325    7032162
1326    7032141
1327    7026014
1328    7014222
1329    7008172
Name: block, dtype: object
New group from main DF, lines 1330 to 1340
Dimension of main DF group:(10, 11)
1330    7014231
1331    7008321
1332    7016011
1333    7003102
1334    7007161
1335    7020003
1336    7012182
1337    7008181
1338    7048051
1339    7024023
Name: block, dtype: object
New group from 

...

8740    7016011
8741    7007161
8742    7009011
8743    7007241
8744    7032141
8745    7003101
8746    7032143
8747    7019002
8748    7023023
8749    7012182
Name: block, dtype: object
New group from main DF, lines 8750 to 8760
Dimension of main DF group:(10, 11)
8750    7032091
8751    7016011
8752    7009041
8753    7001053
8754    7032162
8755    7007191
8756    7002072
8757    7008181
8758    7011012
8759    7016022
Name: block, dtype: object
New group from main DF, lines 8760 to 8770
Dimension of main DF group:(10, 11)
8760    7007212
8761    7001053
8762    7032071
8763    7015083
8764    7007212
8765    7007061
8766    7032133
8767    7007241
8768    7002071
8769    7032011
Name: block, dtype: object
New group from main DF, lines 8770 to 8780
Dimension of main DF group:(10, 11)
8770    7003091
8771    7032132
8772    7024023
8773    7007241
8774    7009041
8775    7009031
8776    7008121
8777    7014172
8778    7007231
8779    7008171
Name: block, dtype: object
New group from 

9050    7007161
9051    7032092
9052    7003062
9053    7012164
9054    7016022
9055    7007113
9056    7007151
9057    7006141
9058    7014173
9059    7016011
Name: block, dtype: object
New group from main DF, lines 9060 to 9070
Dimension of main DF group:(10, 11)
9060    7013142
9061    7009041
9062    7014211
9063    7008182
9064    7038001
9065    7014201
9066    7032141
9067    7015051
9068    7006134
9069    7002072
Name: block, dtype: object
New group from main DF, lines 9070 to 9080
Dimension of main DF group:(10, 11)
9070    7015051
9071    7014173
9072    7032143
9073    7016011
9074    7001041
9075    7008321
9076    7025001
9077    7016022
9078    7007172
9079    7003102
Name: block, dtype: object
New group from main DF, lines 9080 to 9090
Dimension of main DF group:(10, 11)
9080    7006143
9081    7015051
9082    7007193
9083    7016011
9084    7003102
9085    7007192
9086    7014211
9087    7006141
9088    7016011
9089    7051003
Name: block, dtype: object
New group from 

9360    7029002
9361    7006141
9362    7051002
9363    7015031
9364    7007061
9365    7012112
9366    7016022
9367    7008182
9368    7006061
9369    7032141
Name: block, dtype: object
New group from main DF, lines 9370 to 9380
Dimension of main DF group:(10, 11)
9370    7008291
9371    7012011
9372    7008152
9373    7032141
9374    7047002
9375    7008321
9376    7016011
9377    7027004
9378    7027004
9379    7015051
Name: block, dtype: object
New group from main DF, lines 9380 to 9390
Dimension of main DF group:(10, 11)
9380    7008121
9381    7016011
9382    7012021
9383    7008161
9384    7008173
9385    7003091
9386    7006134
9387    7012011
9388    7008173
9389    7014222
Name: block, dtype: object
New group from main DF, lines 9390 to 9400
Dimension of main DF group:(10, 11)
9390    7008162
9391    7032143
9392    7006141
9393    7007212
9394    7018001
9395    7044031
9396    7008152
9397    7007181
9398    7008173
9399    7007212
Name: block, dtype: object
New group from 

9670    7013171
9671    7038001
9672    7007222
9673    7009011
9674    7019001
9675    7048033
9676    7008162
9677    7014232
9678    7008331
9679    7007212
Name: block, dtype: object
New group from main DF, lines 9680 to 9690
Dimension of main DF group:(10, 11)
9680    7008162
9681    7014102
9682    7008221
9683    7003103
9684    7015083
9685    7008184
9686    7032131
9687    7021014
9688    7027001
9689    7007161
Name: block, dtype: object
New group from main DF, lines 9690 to 9700
Dimension of main DF group:(10, 11)
9690    7008134
9691    7019001
9692    7008102
9693    7037011
9694    7032141
9695    7032141
9696    7015051
9697    7019001
9698    7032141
9699    7003092
Name: block, dtype: object
New group from main DF, lines 9700 to 9710
Dimension of main DF group:(10, 11)
9700    7008321
9701    7008261
9702    7009041
9703    7014232
9704    7032132
9705    7024023
9706    7002051
9707    7040003
9708    7032071
9709    7035011
Name: block, dtype: object
New group from 

9980    7008162
9981    7014211
9982    7008321
9983    7014222
9984    7007193
9985    7001052
9986    7015051
9987    7007201
9988    7032071
9989    7032083
Name: block, dtype: object
New group from main DF, lines 9990 to 10000
Dimension of main DF group:(10, 11)
9990    7032141
9991    7017021
9992    7012053
9993    7027001
9994    7007181
9995    7012191
9996    7008173
9997    7032092
9998    7007101
9999    7003103
Name: block, dtype: object


In [28]:
df2.to_csv('/Users/elizabethclick/Desktop/DATA205/df2.csv', index = False)
df2.head()


,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,longitude,latitude,location,block
0,2020-10-25,PRV,Drive,Visitor In Resident Drive,10/27/0020,1801 Hampshire Green Ln,Silver Spring,POINT (-76.974173 39.015179),-76.974173,39.015179,"(39.015179, -76.974173)",8073052
1,2020-10-17,NOP,Henrys,Henrys Back To Lot,10/19/0020,11215 Georgia Ave,Silver Spring,POINT (-77.050954 39.039125),-77.050954,39.039125,"(39.039125, -77.050954)",7038001
2,2020-10-28,NOP,Henrys,Henrys,10/30/0020,10110 Little Pond Pl,Montgomery Village,POINT (-77.208211 39.166813),-77.208211,39.166813,"(39.166813, -77.208211)",7008134
3,2020-10-27,REPO,Other,St Help Financial,10/29/0020,12600 Connecticut Ave,Silver Spring,POINT (-77.073524 39.062209),-77.073524,39.062209,"(39.062209, -77.073524)",7034012
4,2020-10-14,PKG,Prestige,No Prestige,10/16/0020,13802 Castle Blvd,Silver Spring,POINT (-76.943268 39.082785),-76.943268,39.082785,"(39.082785, -76.943268)",7014222


In [29]:
#process third batch (remaining) rows of towing dataframe - get block codes in a new column
df3 = pd.DataFrame()
itld=0
itlu=1
    
for mainDF_chunk in mainDF3:
            nLinesMainCSVd=itld*process_chunksize
            nLinesMainCSVu=itlu*process_chunksize
            print('New group from main DF, lines '+str(nLinesMainCSVd) + ' to ' + str(nLinesMainCSVu))

            
            mainDF_chunk['location']=mainDF_chunk['location'].map(lambda x: eval(str(x)))
            #mainDF_chunk[['tow_date']].apply(pd.to_datetime,errors='coerce')
            print('Dimension of main DF group:' + str(mainDF_chunk.shape))
    
            # Add block from FCC geo to my main dataset in new column "block"
            mainDF_chunk['block']=mainDF_chunk.location.map(lambda x: getblock(x))
            print(mainDF_chunk["block"])
            
            df3 = df3.append(mainDF_chunk)
    
            itlu+=1
            itld+=1

New group from main DF, lines 0 to 10
Dimension of main DF group:(10, 11)
0    7032071
1    7032132
2    7014233
3    7034042
4    7008152
5    7047002
6    7015093
7    7024023
8    7038001
9    7016011
Name: block, dtype: object
New group from main DF, lines 10 to 20
Dimension of main DF group:(10, 11)
10    7007241
11    7015081
12    7032141
13    7014211
14    7019001
15    7015083
16    7032201
17    7019001
18    7032133
19    7003044
Name: block, dtype: object
New group from main DF, lines 20 to 30
Dimension of main DF group:(10, 11)
20    7012113
21    7048033
22    7008221
23    7016011
24    7008301
25    7014211
26    7007192
27    7007173
28    7032131
29    7014222
Name: block, dtype: object
New group from main DF, lines 30 to 40
Dimension of main DF group:(10, 11)
30    7008134
31    7007161
32    7009041
33    7007061
34    7008153
35    7002071
36    7012113
37    7008134
38    7007132
39    7008152
Name: block, dtype: object
New group from main DF, lines 40 to 50
Dime

In [30]:
df3.to_csv('/Users/elizabethclick/Desktop/DATA205/df3.csv', index = False)
df3.head()

,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,longitude,latitude,location,block
0,2021-02-20,NOP,Henrys,Henrys,02/22/0021,2352 Glenmont Cir,Silver Spring,POINT (-77.045818 39.058239),-77.045818,39.058239,"(39.058239, -77.045818)",7032071
1,2021-02-06,NOP,Henrys,Henrys,06/23/0002,3740 Bel Pre Rd,Silver Spring,POINT (-77.074749 39.093468),-77.074749,39.093468,"(39.093468, -77.074749)",7032132
2,2021-01-24,PKG,Speedy,Speedy Towing (),01/26/0021,13901 Carthage Cir,Burtonsville,POINT (-76.936297 39.084801),-76.936297,39.084801,"(39.084801, -76.936297)",7014233
3,2021-02-10,REPO,Thomas Recovery,Thomas Recovery Pd Auto Finance A...,10/23/0002,3509 Edwin St,Silver Spring,POINT (-77.070435 39.053067),-77.070435,39.053067,"(39.053067, -77.070435)",7034042
4,2021-02-03,NOP,Auburn,NaN,03/23/0002,18834 Poppyseed Ln,Germantown,POINT (-77.26894 39.164054),-77.268940,39.164054,"(39.164054, -77.26894)",7008152


In [66]:
frames = [df1, df2, df3]
tow_concat = pd.concat(frames)
tow_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20078 entries, 0 to 77
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tow_date         20069 non-null  object 
 1   tow_reason       20062 non-null  object 
 2   storage_company  19314 non-null  object 
 3   notes            19601 non-null  object 
 4   incident_date    20078 non-null  object 
 5   street           20078 non-null  object 
 6   city             20078 non-null  object 
 7   geo_location     20078 non-null  object 
 8   longitude        20078 non-null  float64
 9   latitude         20078 non-null  float64
 10  location         20078 non-null  object 
 11  block            20078 non-null  object 
dtypes: float64(2), object(10)
memory usage: 2.0+ MB


In [67]:
tow = pd.merge(tow_concat, tractDF, how="inner", left_on="block", right_on="block group", left_index=False, right_index=False,
               sort=True, copy=True, indicator=False, validate=None)
print(tow.head())
tow.info()

     tow_date tow_reason     storage_company  \
0  2020-12-11       REPO        Final Notice   
1  2021-02-17       REPO                 NaN   
2  2021-02-10       REPO              Thomas   
3  2020-02-15       REPO  Greenwood Recovery   
4  2020-02-04       REPO  Greenwood Recovery   

                                               notes incident_date  \
0   Financial Comp Gm Finance     Owner Dan Stanback    11/22/0012   
1  Tow Co Greenwood Recovery         Finance Co  ...    02/19/0021   
2                 Reason  Santander  Thomas Recovery    10/23/0002   
3                             Chase Auto   Greenwood    02/15/2020   
4  Mountain America Credit Union   Greenwood Reco...    02/04/2020   

                  street          city                  geo_location  \
0    21704 Stableview Dr  Gaithersburg  POINT (-77.175624 39.211873)   
1          8212 Brink Rd  Gaithersburg  POINT (-77.167752 39.208712)   
2          7321 Brink Rd  Gaithersburg  POINT (-77.148754 39.211605)   
3 

In [68]:
#Final cleaning step - drop columns not needed anymore "notes", "incident_date", street" etc.
print(tow.info())
tow.drop('notes', axis=1, inplace=True) 
tow.drop('incident_date', axis=1, inplace=True)
tow.drop('NAME', axis = 1, inplace=True)
#tow.drop('state_y', axis=1, inplace=True)
tow.drop('block group', axis=1, inplace=True)
tow.drop('county', axis=1, inplace=True)
tow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19975 entries, 0 to 19974
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tow_date         19966 non-null  object 
 1   tow_reason       19959 non-null  object 
 2   storage_company  19215 non-null  object 
 3   notes            19501 non-null  object 
 4   incident_date    19975 non-null  object 
 5   street           19975 non-null  object 
 6   city             19975 non-null  object 
 7   geo_location     19975 non-null  object 
 8   longitude        19975 non-null  float64
 9   latitude         19975 non-null  float64
 10  location         19975 non-null  object 
 11  block            19975 non-null  object 
 12  NAME             19975 non-null  object 
 13  hh_income        19975 non-null  object 
 14  pop_total        19975 non-null  object 
 15  pop_black        19975 non-null  object 
 16  state            19975 non-null  object 
 17  county      

In [69]:
tow.to_csv('/Users/elizabethclick/Desktop/DATA205/tow_clean.csv', index = False)
tow.head()

,tow_date,tow_reason,storage_company,street,city,geo_location,longitude,latitude,location,block,hh_income,pop_total,pop_black,state,tract
0,2020-12-11,REPO,Final Notice,21704 Stableview Dr,Gaithersburg,POINT (-77.175624 39.211873),-77.175624,39.211873,"(39.211873, -77.175624)",7001012,208929,1931,39,24,700101
1,2021-02-17,REPO,NaN,8212 Brink Rd,Gaithersburg,POINT (-77.167752 39.208712),-77.167752,39.208712,"(39.208712, -77.167752)",7001013,135208,800,10,24,700101
2,2021-02-10,REPO,Thomas,7321 Brink Rd,Gaithersburg,POINT (-77.148754 39.211605),-77.148754,39.211605,"(39.211605, -77.148754)",7001013,135208,800,10,24,700101
3,2020-02-15,REPO,Greenwood Recovery,21400 Laytonsville Rd,Gaithersburg,POINT (-77.141329 39.207148),-77.141329,39.207148,"(39.207148, -77.141329)",7001032,218173,1017,193,24,700103
4,2020-02-04,REPO,Greenwood Recovery,7571 Lindbergh Dr,Gaithersburg,POINT (-77.153158 39.166038),-77.153158,39.166038,"(39.166038, -77.153158)",7001033,172778,3112,345,24,700103


In [70]:
tow = pd.read_csv('/Users/elizabethclick/Desktop/DATA205/tow_clean.csv')
tow.head()

,tow_date,tow_reason,storage_company,street,city,geo_location,longitude,latitude,location,block,hh_income,pop_total,pop_black,state,tract
0,2020-12-11,REPO,Final Notice,21704 Stableview Dr,Gaithersburg,POINT (-77.175624 39.211873),-77.175624,39.211873,"(39.211873, -77.175624)",7001012,208929,1931,39,24,700101
1,2021-02-17,REPO,NaN,8212 Brink Rd,Gaithersburg,POINT (-77.167752 39.208712),-77.167752,39.208712,"(39.208712, -77.167752)",7001013,135208,800,10,24,700101
2,2021-02-10,REPO,Thomas,7321 Brink Rd,Gaithersburg,POINT (-77.148754 39.211605),-77.148754,39.211605,"(39.211605, -77.148754)",7001013,135208,800,10,24,700101
3,2020-02-15,REPO,Greenwood Recovery,21400 Laytonsville Rd,Gaithersburg,POINT (-77.141329 39.207148),-77.141329,39.207148,"(39.207148, -77.141329)",7001032,218173,1017,193,24,700103
4,2020-02-04,REPO,Greenwood Recovery,7571 Lindbergh Dr,Gaithersburg,POINT (-77.153158 39.166038),-77.153158,39.166038,"(39.166038, -77.153158)",7001033,172778,3112,345,24,700103


In [71]:
#CONNECT last chunk to CAPSTONE ANALYSIS and VISUALIZATION